MultiLayer Perceptron (MLP): O método Perceptron Multicamada teve sua origem com os estudos do matemático Walter Pitts em conjunto com o neurocientista Warren McCulloch criaram o primeiro circuito (neurônio de McCulloch-Pitts) objetivando simular um neurônio biológico em 1943, mas foi com os estudos de Frank Rosenblatt que em 1958 foi publicado o artigo sobre Perceptron sendo o primeiro neurônio artificial (capaz de resolver somente problemas lineares) que serviu de base para o Adaline e posteriormente ao MLP. Somente em 1980 depois de muitas pesquisas realizadas pelos pesquisadores David Rumelhart, Geoffrey Hinton, Ronald Williams é que foi publicado o trabalho sobre o Multilayer Perceptron (MLP) que passou a resolver problemas não lineares, fortalecendo o conceito de Machine Learning como base na arquitetura das Redes Neurais Artificiais (RNA), essa sendo totalmente conectada (full-conect) implementando o método denominado de backpropagation em uma rede neural feedforward.

Rosenblatt, F. The perceptron: a probabilistic model for information storage and organization in the brain. Psychological Review, 65(6), 386-408. https://doi.org/10.1037/h0042519, 1958.

Explicação do algoritmo MLP:

In [ ]:
#instalar biblioteca Orange Canvas
!pip install Orange3

In [ ]:
#importar biblioteca Orange Canvas
import Orange

In [ ]:
#importar dados do disco local
from google.colab import files  
files.upload()

In [ ]:
#instanciar objeto de dados com base no caminho gerado com a importação do arquivo
dados = Orange.data.Table("/content/simulacao1.csv")

In [ ]:
#exibir os primeiros 5 registros para uma rápida análise da importação dos dados
for d in dados[:5]:
  print(d)

In [ ]:
#explorar os metadados da base de dados importada
qtde_campos = len(dados.domain.attributes)
qtde_cont = sum(1 for a in dados.domain.attributes if a.is_continuous)
qtde_disc = sum(1 for a in dados.domain.attributes if a.is_discrete)
print("%d metadados: %d continuos, %d discretos" % (qtde_campos, qtde_cont, qtde_disc))
print("Nome dos metadados:", ", ".join(dados.domain.attributes[i].name for i in range(qtde_campos)),)

In [ ]:
#explorar domínios dos atributos (campos da base de dados)
dados.domain.attributes

In [ ]:
#explorar as classes da base de dados
dados.domain.class_var

In [ ]:
#explorar dados (quantidade de campos e registros da base de dados)
print("Campos:", ", ".join(c.name for c in dados.domain.attributes))
print("Registros:", len(dados))
print("Valor do registro 1 da coluna 1:", dados[0][0])
print("Valor do registro 2 da coluna 2:", dados[1][1])

In [ ]:
#criar amostra de dados, sendo 70% para treinamento e teste e 30% para avaliação
qtde100 = len(dados)
qtde70 = len(dados) * 70 / 100
qtde30 = len(dados) * 30 / 100
print("Qtde 100%:", qtde100)
print("Qtde  70%:", qtde70)
print("Qtde  30%:", qtde30)
amostra = Orange.data.Table(dados.domain, [d for d in dados if d.row_index < qtde70])
print("Registros:", len(dados))
print("Registros:", len(amostra))

**Hiperparâmetros do Multilayer Perceptron**

hidden_layer_sizes=(100,),

activation='relu',

solver='adam',

alpha=0.0001,

batch_size='auto',

learning_rate='constant',

learning_rate_init=0.001,

power_t=0.5, max_iter=200,

shuffle=True,

random_state=None,

tol=0.0001, verbose=False,

warm_start=False, momentum=0.9,

nesterovs_momentum=True,

early_stopping=False,

validation_fraction=0.1,

beta_1=0.9,

beta_2=0.999,

epsilon=1e-08,

max_iter: número máximo de iterações.

preprocessors: é usado o pré-processamento padrão quando nenhum outro pré-processador é fornecido. Ele os executa na seguinte ordem: remove instâncias com valores de destino desconhecidos; continua variáveis categóricas (com codificação one-hot); remove colunas vazias; imputa valores ausentes com valores médios.

In [ ]:
#inicializar a instância com os seus hiperparâmetros para a técnica Multilayer Perceptron (MLP) Cenário1
mlp = Orange.classification.NNClassificationLearner(
    hidden_layer_sizes=(100,),
    activation='relu',
    solver='adam',
    alpha=0.0001,
    batch_size='auto',
    learning_rate='constant',
    learning_rate_init=0.001,
    power_t=0.5, max_iter=200,
    shuffle=True,
    random_state=None,
    tol=0.0001, verbose=False,
    warm_start=False, momentum=0.9,
    nesterovs_momentum=True,
    early_stopping=False,
    validation_fraction=0.1,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-08,
    preprocessors=None)

In [ ]:
#ligar técnica MLP com os dados da simulação
dados_mlp = mlp(dados)

In [ ]:
#treinar e testar técnica MLP com os dados com 5 partes (folders)
avalia_mlp = Orange.evaluation.CrossValidation(dados, [mlp], k=5)

In [ ]:
#avaliar os indicadores para o SVM com base nas métricas de classificação
print("Acurácia:  %.3f" % Orange.evaluation.scoring.CA(avalia_mlp)[0])
print("Precisão:  %.3f" % Orange.evaluation.scoring.Precision(avalia_mlp)[0])
print("Revocação: %.3f" % Orange.evaluation.scoring.Recall(avalia_mlp)[0])
print("F1:        %.3f" % Orange.evaluation.scoring.F1(avalia_mlp)[0])
print("ROC:       %.3f" % Orange.evaluation.scoring.AUC(avalia_mlp)[0])

In [ ]:
#inicializar a instância com os seus hiperparâmetros para a técnica Supporte Vector Machine (SVM) Cenário2
svm = Orange.classification.SVMLearner(
    C=1.0, 
    kernel='rbf', 
    degree=3, 
    gamma='auto', 
    coef0=0.0, 
    shrinking=True, 
    probability=False, 
    tol=0.001, 
    cache_size=200, 
    max_iter=2000, 
    preprocessors=None)

In [ ]:
#inicializar a instância com os seus hiperparâmetros para a técnica K-Nearest Neighbors (KNN) Cenário3
knn = Orange.classification.KNNLearner(
    n_neighbors=5, 
    metric='euclidean', 
    weights='distance', 
    algorithm='auto', 
    metric_params=None, 
    preprocessors=None)

In [ ]:
#validar o aprendizado para as 3 técnicas
aprendizado = [mlp, svm, knn]
avaliacao = Orange.evaluation.CrossValidation(dados, aprendizado, k=5)

In [ ]:
#imprimir os indicadores para as 3 técnicas
print(" " * 10 + " | ".join("%-4s" % learner.name for learner in aprendizado))
print("Acurácia  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.CA(avaliacao)))
print("Precisão  %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Precision(avaliacao)))
print("Revocação %s" % " | ".join("%.2f" % s for s in Orange.evaluation.Recall(avaliacao)))
print("F1        %s" % " | ".join("%.2f" % s for s in Orange.evaluation.F1(avaliacao)))
print("ROC       %s" % " | ".join("%.2f" % s for s in Orange.evaluation.AUC(avaliacao)))